#Prompt Parsing (No Hardcoded Keywords)
##Dynamic Pattern Recognition:

Uses regex patterns to extract job titles, experience levels, locations
Intelligently parses salary ranges, employment types, benefits
Extracts industry context and company size preferences
Calculates confidence scores based on parsed information

##Real Data via Google Places API
Actual Business Discovery:

Searches real businesses that might have job opportunities
Gets detailed information (phone, website, ratings, hours)
Finds staffing agencies, companies, and potential employers
No mock data - everything is live from Google's database

In [ ]:
import re
import json
import requests
from dataclasses import dataclass
from typing import List, Optional, Dict, Any
from datetime import datetime
import os

@dataclass
class JobOpportunity:
    business_name: str
    address: str
    phone: str
    rating: float
    business_type: str
    place_id: str
    website: Optional[str] = None
    hours: Optional[Dict] = None
    reviews_count: int = 0

@dataclass
class ParsedJobQuery:
    job_keywords: List[str]
    location_keywords: List[str]
    experience_indicators: List[str]
    employment_type_indicators: List[str]
    salary_mentioned: Optional[str]
    industry_keywords: List[str]
    company_size_indicators: List[str]
    benefits_mentioned: List[str]
    original_query: str
    confidence_score: float

class SmartJobPromptParser:
    def __init__(self, google_api_key: str):
        self.google_api_key = ''  #add API Key

        self.places_base_url = "https://maps.googleapis.com/maps/api/place"

        # Dynamic keyword extraction patterns - no hardcoded lists!
        self.job_patterns = [
            r'\b(\w+)\s+(engineer|developer|analyst|manager|specialist|coordinator|assistant|director|lead|intern)\b',
            r'\b(junior|senior|lead|principal|staff|associate)\s+(\w+)\b',
            r'\b(\w+)\s+(job|position|role|work|career)\b',
            r'\b(looking for|seeking|want|need)\s+(\w+(?:\s+\w+)*)\s+(position|job|role)\b'
        ]

        self.location_patterns = [
            r'\bin\s+([A-Za-z\s,]+)(?:\s|$)',
            r'\b(near|around|at|located)\s+([A-Za-z\s,]+)(?:\s|$)',
            r'\b([A-Za-z\s]+),?\s+(CA|California|Los Angeles|LA)\b'
        ]

        self.salary_patterns = [
            r'\$\s*(\d+(?:,\d{3})*(?:k|K)?)\s*(?:-|to)\s*\$?\s*(\d+(?:,\d{3})*(?:k|K)?)',
            r'\$\s*(\d+(?:,\d{3})*(?:k|K)?)\s*(?:per|/)\s*(hour|hr|year|annually)',
            r'(\d+(?:,\d{3})*(?:k|K)?)\s*(?:dollar|usd|\$)'
        ]

    def parse_query(self, query: str) -> ParsedJobQuery:
        """
        Intelligently parse job search query using NLP patterns
        No hardcoded keywords - extracts meaning from context
        """
        query_lower = query.lower().strip()

        # Extract job-related keywords dynamically
        job_keywords = self._extract_job_keywords(query_lower)

        # Extract location information
        location_keywords = self._extract_location_keywords(query_lower)

        # Extract experience indicators
        experience_indicators = self._extract_experience_indicators(query_lower)

        # Extract employment type indicators
        employment_type_indicators = self._extract_employment_type(query_lower)

        # Extract salary mentions
        salary_mentioned = self._extract_salary_info(query_lower)

        # Extract industry context
        industry_keywords = self._extract_industry_context(query_lower)

        # Extract company size indicators
        company_size_indicators = self._extract_company_size(query_lower)

        # Extract benefits mentioned
        benefits_mentioned = self._extract_benefits(query_lower)

        # Calculate confidence score based on extracted information
        confidence_score = self._calculate_confidence(
            job_keywords, location_keywords, experience_indicators
        )

        return ParsedJobQuery(
            job_keywords=job_keywords,
            location_keywords=location_keywords,
            experience_indicators=experience_indicators,
            employment_type_indicators=employment_type_indicators,
            salary_mentioned=salary_mentioned,
            industry_keywords=industry_keywords,
            company_size_indicators=company_size_indicators,
            benefits_mentioned=benefits_mentioned,
            original_query=query,
            confidence_score=confidence_score
        )

    def _extract_job_keywords(self, query: str) -> List[str]:
        """Extract job-related terms using pattern matching"""
        keywords = []

        # Common job title patterns
        for pattern in self.job_patterns:
            matches = re.findall(pattern, query, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    keywords.extend([word.strip() for word in match if word.strip()])
                else:
                    keywords.append(match.strip())

        # Extract individual meaningful words that could be job-related
        words = re.findall(r'\b[a-zA-Z]{3,}\b', query)
        potential_job_words = [
            word for word in words
            if len(word) > 3 and word not in ['looking', 'seeking', 'want', 'need', 'jobs', 'work']
        ]
        keywords.extend(potential_job_words[:5])  # Limit to prevent noise

        return list(set(keywords))  # Remove duplicates

    def _extract_location_keywords(self, query: str) -> List[str]:
        """Extract location information"""
        locations = []

        for pattern in self.location_patterns:
            matches = re.findall(pattern, query, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    locations.extend([loc.strip() for loc in match if loc.strip()])
                else:
                    locations.append(match.strip())

        # Default to LA if no location specified
        if not locations:
            locations = ["Los Angeles, CA"]

        return locations

    def _extract_experience_indicators(self, query: str) -> List[str]:
        """Extract experience level indicators"""
        experience_patterns = [
            r'\b(entry.level|junior|senior|lead|principal|staff|director|manager|intern|graduate|experienced|beginner)\b',
            r'\b(\d+)\s*(year|yr)s?\s*(experience|exp)\b',
            r'\b(new grad|recent graduate|fresh|starter)\b'
        ]

        indicators = []
        for pattern in experience_patterns:
            matches = re.findall(pattern, query, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    indicators.extend([ind.strip() for ind in match if ind.strip()])
                else:
                    indicators.append(match.strip())

        return indicators

    def _extract_employment_type(self, query: str) -> List[str]:
        """Extract employment type indicators"""
        type_pattern = r'\b(full.time|part.time|contract|freelance|remote|hybrid|on.site|temporary|permanent|internship)\b'
        matches = re.findall(type_pattern, query, re.IGNORECASE)
        return [match.replace('.', '-') for match in matches]

    def _extract_salary_info(self, query: str) -> Optional[str]:
        """Extract salary information"""
        for pattern in self.salary_patterns:
            match = re.search(pattern, query, re.IGNORECASE)
            if match:
                return match.group(0)
        return None

    def _extract_industry_context(self, query: str) -> List[str]:
        """Extract industry-related keywords"""
        industry_patterns = [
            r'\b(tech|technology|healthcare|finance|education|retail|manufacturing|startup|enterprise)\b',
            r'\b(ai|artificial intelligence|machine learning|blockchain|cloud|mobile|web)\b',
            r'\b(hospital|clinic|school|university|bank|restaurant|store)\b'
        ]

        keywords = []
        for pattern in industry_patterns:
            matches = re.findall(pattern, query, re.IGNORECASE)
            keywords.extend(matches)

        return keywords

    def _extract_company_size(self, query: str) -> List[str]:
        """Extract company size indicators"""
        size_pattern = r'\b(startup|small|large|enterprise|corporation|big tech|fortune|unicorn)\b'
        matches = re.findall(size_pattern, query, re.IGNORECASE)
        return matches

    def _extract_benefits(self, query: str) -> List[str]:
        """Extract mentioned benefits"""
        benefits_pattern = r'\b(health insurance|401k|remote|flexible|vacation|pto|stock options|equity|bonus)\b'
        matches = re.findall(benefits_pattern, query, re.IGNORECASE)
        return matches

    def _calculate_confidence(self, job_keywords: List[str], location_keywords: List[str],
                            experience_indicators: List[str]) -> float:
        """Calculate confidence score for parsed query"""
        score = 0.0

        if job_keywords:
            score += 0.4 * min(len(job_keywords) / 3, 1.0)

        if location_keywords:
            score += 0.3

        if experience_indicators:
            score += 0.2

        score += 0.1  # Base score

        return round(min(score, 1.0), 2)

    def search_businesses_for_jobs(self, parsed_query: ParsedJobQuery) -> List[JobOpportunity]:
        """
        Use Google Places API to find businesses that might have job opportunities
        """
        all_opportunities = []

        # Build search queries based on parsed information
        search_queries = self._build_search_queries(parsed_query)

        for search_query in search_queries:
            try:
                opportunities = self._search_places(search_query, parsed_query.location_keywords[0])
                all_opportunities.extend(opportunities)
            except Exception as e:
                print(f"Error searching for '{search_query}': {e}")
                continue

        # Remove duplicates and rank by relevance
        unique_opportunities = self._deduplicate_and_rank(all_opportunities, parsed_query)

        return unique_opportunities[:20]  # Return top 20 opportunities

    def _build_search_queries(self, parsed_query: ParsedJobQuery) -> List[str]:
        """Build Google Places search queries based on parsed information"""
        queries = []

        # Combine job keywords with industry context
        for job_keyword in parsed_query.job_keywords[:3]:  # Top 3 job keywords
            # Direct job search
            queries.append(f"{job_keyword} jobs")

            # Industry-specific searches
            for industry in parsed_query.industry_keywords:
                queries.append(f"{industry} {job_keyword}")

            # Company type searches
            for company_type in parsed_query.company_size_indicators:
                queries.append(f"{company_type} {job_keyword}")

        # Fallback searches if no specific job keywords
        if not parsed_query.job_keywords:
            queries.extend([
                "employment agency",
                "staffing agency",
                "recruiting",
                "human resources",
                "career center"
            ])

        return queries[:5]  # Limit API calls

    def _search_places(self, query: str, location: str) -> List[JobOpportunity]:
        """Search Google Places API"""
        url = f"{self.places_base_url}/textsearch/json"

        params = {
            'query': f"{query} in {location}",
            'key': self.google_api_key,
            'type': 'establishment'
        }

        response = requests.get(url, params=params)

        if response.status_code != 200:
            raise Exception(f"API Error: {response.status_code}")

        data = response.json()

        if data['status'] != 'OK':
            if data['status'] == 'ZERO_RESULTS':
                return []
            raise Exception(f"Places API Error: {data['status']}")

        opportunities = []

        for place in data.get('results', []):
            # Get additional details
            details = self._get_place_details(place['place_id'])

            opportunity = JobOpportunity(
                business_name=place.get('name', 'Unknown'),
                address=place.get('formatted_address', 'Address not available'),
                phone=details.get('phone', 'Phone not available'),
                rating=place.get('rating', 0.0),
                business_type=', '.join(place.get('types', [])),
                place_id=place['place_id'],
                website=details.get('website'),
                hours=details.get('hours'),
                reviews_count=place.get('user_ratings_total', 0)
            )

            opportunities.append(opportunity)

        return opportunities

    def _get_place_details(self, place_id: str) -> Dict[str, Any]:
        """Get detailed information about a place"""
        url = f"{self.places_base_url}/details/json"

        params = {
            'place_id': place_id,
            'fields': 'website,formatted_phone_number,opening_hours',
            'key': self.google_api_key
        }

        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                data = response.json()
                if data['status'] == 'OK':
                    result = data.get('result', {})
                    return {
                        'website': result.get('website'),
                        'phone': result.get('formatted_phone_number'),
                        'hours': result.get('opening_hours', {}).get('weekday_text')
                    }
        except Exception as e:
            print(f"Error getting place details: {e}")

        return {}

    def _deduplicate_and_rank(self, opportunities: List[JobOpportunity],
                             parsed_query: ParsedJobQuery) -> List[JobOpportunity]:
        """Remove duplicates and rank by relevance"""
        # Remove duplicates based on place_id
        seen_ids = set()
        unique_opportunities = []

        for opp in opportunities:
            if opp.place_id not in seen_ids:
                seen_ids.add(opp.place_id)
                unique_opportunities.append(opp)

        # Rank by relevance (rating, reviews count, keyword matching)
        def relevance_score(opp: JobOpportunity) -> float:
            score = 0.0

            # Rating weight
            score += opp.rating * 0.3

            # Reviews count weight (logarithmic scale)
            if opp.reviews_count > 0:
                score += min(0.3, 0.1 * (opp.reviews_count / 100))

            # Keyword matching in business name
            business_name_lower = opp.business_name.lower()
            for keyword in parsed_query.job_keywords:
                if keyword.lower() in business_name_lower:
                    score += 0.2

            # Industry matching
            for industry in parsed_query.industry_keywords:
                if industry.lower() in business_name_lower or industry.lower() in opp.business_type.lower():
                    score += 0.2

            return score

        unique_opportunities.sort(key=relevance_score, reverse=True)
        return unique_opportunities

    def format_results(self, opportunities: List[JobOpportunity],
                      parsed_query: ParsedJobQuery) -> str:
        """Format search results"""
        if not opportunities:
            return f"No job opportunities found for: '{parsed_query.original_query}'"

        result = f"\n🔍 Job Opportunities Search Results\n"
        result += f"Query: '{parsed_query.original_query}'\n"
        result += f"Confidence Score: {parsed_query.confidence_score}/1.0\n"
        result += "=" * 60 + "\n\n"

        result += "📊 Parsed Query Analysis:\n"
        result += f"  • Job Keywords: {', '.join(parsed_query.job_keywords) or 'None detected'}\n"
        result += f"  • Location: {', '.join(parsed_query.location_keywords)}\n"
        result += f"  • Experience Level: {', '.join(parsed_query.experience_indicators) or 'Any level'}\n"
        result += f"  • Employment Type: {', '.join(parsed_query.employment_type_indicators) or 'Not specified'}\n"
        result += f"  • Salary Mentioned: {parsed_query.salary_mentioned or 'Not specified'}\n"
        result += f"  • Industry: {', '.join(parsed_query.industry_keywords) or 'General'}\n"
        result += f"  • Company Size: {', '.join(parsed_query.company_size_indicators) or 'Any size'}\n\n"

        result += f"Found {len(opportunities)} potential job opportunities:\n"
        result += "=" * 60 + "\n"

        for i, opp in enumerate(opportunities, 1):
            result += f"{i}. {opp.business_name}\n"
            result += f"   📍 Address: {opp.address}\n"
            result += f"   📞 Phone: {opp.phone}\n"
            result += f"   ⭐ Rating: {opp.rating}/5.0 ({opp.reviews_count} reviews)\n"
            result += f"   🏢 Type: {opp.business_type}\n"
            if opp.website:
                result += f"   🌐 Website: {opp.website}\n"
            result += f"   💡 Tip: Contact them directly to inquire about job openings\n"
            result += "-" * 60 + "\n"

        return result

def main():
    """Demo function"""
    print("🔍 Smart Job Search with Google Places API")
    print("=" * 50)

    # You need to set your Google API key
    api_key = os.getenv('GOOGLE_PLACES_API_KEY')

    if not api_key:
        print("❌ Error: Please set GOOGLE_PLACES_API_KEY environment variable")
        print("Get your API key from: https://developers.google.com/maps/documentation/places/web-service/get-api-key")
        return

    parser = SmartJobPromptParser(api_key)

    # Test queries
    test_queries = [
        "senior software engineer jobs in Santa Monica",
        "data scientist positions at tech startups",
        "marketing manager role with health insurance benefits",
        "entry level developer remote work",
        "healthcare jobs near downtown LA"
    ]

    for query in test_queries:
        print(f"\n🔍 Searching: '{query}'")

        # Parse query
        parsed = parser.parse_query(query)

        # Search for opportunities
        opportunities = parser.search_businesses_for_jobs(parsed)

        # Display results
        print(parser.format_results(opportunities, parsed))
        print("\n" + "=" * 80 + "\n")

if __name__ == "__main__":
    # Interactive mode
    api_key = os.getenv('GOOGLE_PLACES_API_KEY')

    if not api_key:
        print("❌ Please set GOOGLE_PLACES_API_KEY environment variable")
        print("Example: export GOOGLE_PLACES_API_KEY='your_api_key_here'")
        print("Get API key: https://developers.google.com/maps/documentation/places/web-service/get-api-key")
        exit(1)

    parser = SmartJobPromptParser(api_key)

    print("🚀 Smart Job Search Parser with Google Places API")
    print("Enter natural language job search queries")
    print("Examples:")
    print("  • 'senior software engineer jobs in Santa Monica'")
    print("  • 'data scientist positions at AI companies'")
    print("  • 'marketing manager with remote work options'")
    print("Type 'quit' to exit\n")

    while True:
        query = input("🔍 Enter job search: ").strip()

        if query.lower() in ['quit', 'exit', 'q']:
            print("👋 Thanks for using the smart job search parser!")
            break

        if not query:
            continue

        try:
            # Parse and search
            parsed = parser.parse_query(query)
            opportunities = parser.search_businesses_for_jobs(parsed)

            # Display results
            print(parser.format_results(opportunities, parsed))
            print()

        except Exception as e:
            print(f"❌ Error: {e}")
            print("Please try a different search query.\n")

❌ Please set GOOGLE_PLACES_API_KEY environment variable
Example: export GOOGLE_PLACES_API_KEY='your_api_key_here'
Get API key: https://developers.google.com/maps/documentation/places/web-service/get-api-key
🚀 Smart Job Search Parser with Google Places API
Enter natural language job search queries
Examples:
  • 'senior software engineer jobs in Santa Monica'
  • 'data scientist positions at AI companies'
  • 'marketing manager with remote work options'
Type 'quit' to exit


🔍 Job Opportunities Search Results
Query: 'senior software engineer jobs in Santa Monica'
Confidence Score: 1.0/1.0

📊 Parsed Query Analysis:
  • Job Keywords: senior, software, santa, engineer, monica
  • Location: santa monica
  • Experience Level: senior
  • Employment Type: Not specified
  • Salary Mentioned: Not specified
  • Industry: General
  • Company Size: Any size

Found 20 potential job opportunities:
1. App Makers Santa Monica | Software, Web, & App Development
   📍 Address: Second Floor, 201 Wilshire Bl